## Modelo simple utilizando Word Embedding y LSTM

Esta notebook presenta un modelo sencillo de **Deep Neural Network** que tiene como una capa que utiliza un *corpus* de ***word embeddings*** pre-entrenados y otra capa denominada **LSTM** que son muy utilizada en el área de **NLP**.

Si bien, se trata de un dataset pequeño, resulta interesante ver como se comporta dicho modelo.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.utils import to_categorical
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
import gensim

## Lectura de los datos

In [4]:
import pandas as pd

df_train = pd.read_csv('training_data_preprocessed.csv')
df_train.head()

,review,score,review cleaned,reviews_length,number_of_words,reviews_avg_length,review cleaned stopwords,lemmatized
0,Era necesario mucho coraje para abordar aconte...,buena,era necesario mucho coraje para abordar aconte...,3812,649,4.873652,era necesario mucho coraje para abordar aconte...,necesario coraje abordar acontecimiento recien...
1,Esperaba con curiosidad y ciertas ganas el est...,mala,esperaba con curiosidad y ciertas ganas el est...,2259,405,4.577778,esperaba con curiosidad y ciertas ganas el est...,esperar curiosidad y gana estreno antonio band...
2,"Wes Craven, convertido en factoría, nos vuelve...",mala,wes craven convertido en factoría nos vuelve a...,1816,317,4.728707,wes craven convertido en factoría nos vuelve a...,wes cravir convertido factoría volver a contar...
3,Va la gente y se rasga las vestiduras con 'Caó...,mala,va la gente y se rasga las vestiduras con caót...,3598,624,4.766026,va la gente y se rasga las vestiduras con caót...,gente y rasgar vestidura caótico án julio mede...
4,Director: Mariano Ozores.Duración: 77 minutos....,buena,director mariano ozoresduración 77 minutosestr...,2271,395,4.749367,mariano ozoresduración minutosestreno de dic...,mariano ozoresduración minutosestreno dici...


**Max len**

In [5]:
reviews = df_train['review cleaned stopwords']
reviews_split = reviews.str.split()
len_reviews = reviews_split.apply(lambda x : len(x))
df_train['len reviews'] = len_reviews
df_train.head()

,review,score,review cleaned,reviews_length,number_of_words,reviews_avg_length,review cleaned stopwords,lemmatized,len reviews
0,Era necesario mucho coraje para abordar aconte...,buena,era necesario mucho coraje para abordar aconte...,3812,649,4.873652,era necesario mucho coraje para abordar aconte...,necesario coraje abordar acontecimiento recien...,606
1,Esperaba con curiosidad y ciertas ganas el est...,mala,esperaba con curiosidad y ciertas ganas el est...,2259,405,4.577778,esperaba con curiosidad y ciertas ganas el est...,esperar curiosidad y gana estreno antonio band...,379
2,"Wes Craven, convertido en factoría, nos vuelve...",mala,wes craven convertido en factoría nos vuelve a...,1816,317,4.728707,wes craven convertido en factoría nos vuelve a...,wes cravir convertido factoría volver a contar...,307
3,Va la gente y se rasga las vestiduras con 'Caó...,mala,va la gente y se rasga las vestiduras con caót...,3598,624,4.766026,va la gente y se rasga las vestiduras con caót...,gente y rasgar vestidura caótico án julio mede...,601
4,Director: Mariano Ozores.Duración: 77 minutos....,buena,director mariano ozoresduración 77 minutosestr...,2271,395,4.749367,mariano ozoresduración minutosestreno de dic...,mariano ozoresduración minutosestreno dici...,369


**Encode categories**

In [6]:
from sklearn.preprocessing import LabelEncoder

lb_category = LabelEncoder()
df_train['label'] = lb_category.fit_transform(df_train['score'])
df_train.head()

,review,score,review cleaned,reviews_length,number_of_words,reviews_avg_length,review cleaned stopwords,lemmatized,len reviews,label
0,Era necesario mucho coraje para abordar aconte...,buena,era necesario mucho coraje para abordar aconte...,3812,649,4.873652,era necesario mucho coraje para abordar aconte...,necesario coraje abordar acontecimiento recien...,606,0
1,Esperaba con curiosidad y ciertas ganas el est...,mala,esperaba con curiosidad y ciertas ganas el est...,2259,405,4.577778,esperaba con curiosidad y ciertas ganas el est...,esperar curiosidad y gana estreno antonio band...,379,1
2,"Wes Craven, convertido en factoría, nos vuelve...",mala,wes craven convertido en factoría nos vuelve a...,1816,317,4.728707,wes craven convertido en factoría nos vuelve a...,wes cravir convertido factoría volver a contar...,307,1
3,Va la gente y se rasga las vestiduras con 'Caó...,mala,va la gente y se rasga las vestiduras con caót...,3598,624,4.766026,va la gente y se rasga las vestiduras con caót...,gente y rasgar vestidura caótico án julio mede...,601,1
4,Director: Mariano Ozores.Duración: 77 minutos....,buena,director mariano ozoresduración 77 minutosestr...,2271,395,4.749367,mariano ozoresduración minutosestreno de dic...,mariano ozoresduración minutosestreno dici...,369,0


In [7]:
X = df_train['review cleaned stopwords'].values
y = df_train['label']
classes = df_train['label'].unique().shape[0]
maxLen = df_train['len reviews'].max() + 1
classes

2

**Generación del dataset de entrenamiento y de test**

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

## Lectura del embeddings pre-entrenados

Este modelo utiliza un modelo de ***word embeddings*** pre-entrenados que puede ser descargado aquí: https://www.kaggle.com/rtatman/pretrained-word-vectors-for-spanish

In [8]:
word2vect_model = KeyedVectors.load_word2vec_format('./word_embedding_pre_trained/SBW-vectors-300-min5.txt', binary=False)

In [12]:
vocab = list(word2vect_model.vocab.keys())

In [10]:
word2vect_model.vector_size

300

## Definición de la arquitectura de la Red Neuronal

Aquí se define la arquitectura de la Red Neuronal. Se trata de una arquitectura sencilla, con una capa que utiliza ***word embeddings*** pre-entrenados y una capa **LSTM**. 

In [20]:
def sentences_to_indices(X, vocab, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()`  
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    # Initialize X_indices as a numpy matrix of zeros and the correct shape
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if w in vocab:
                X_indices[i, j] = vocab.index(w)
            # Increment j to j + 1
            j = j+1
    
    return X_indices

In [21]:
def pretrained_embedding_layer(word2vect_model, vocab, maxLen):
    """
    Creates a Keras Embedding() layer and loads in pre-trained 300-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their word embedding vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(vocab) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word2vect_model.vector_size      # define dimensionality of your GloVe word vectors
    
    # Initialize the embedding matrix as a numpy array of zeros of shape
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    index = 0
    for word in vocab:
        emb_matrix[index, :] = word2vect_model.get_vector(word=word)
        index +=1

    # Define Keras embedding layer 
    embedding_layer = Embedding(input_dim=vocab_len, output_dim= emb_dim, trainable=False, input_length=maxLen)

    # Build the embedding layer, it is required before setting the weights of the embedding layer.
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [22]:
def get_model(input_shape, word2vect_model, vocab, classes):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    model = Sequential()
    #model.add(Input(shape=input_shape, dtype='int32'))
    embedding_layer = pretrained_embedding_layer(word2vect_model, vocab, input_shape[0])
    model.add(embedding_layer)
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    #model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(classes, activation='softmax'))
    
    return model

In [23]:
model = get_model((maxLen,), word2vect_model, vocab, classes)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4380, 300)         300196200 
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 300,416,106
Trainable params: 219,906
Non-trainable params: 300,196,200
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train_indices = sentences_to_indices(X_train, vocab, maxLen)

In [ ]:
one_hot_labels = to_categorical(y_train, num_classes=classes)

In [ ]:
model.fit(X_train_indices, one_hot_labels, epochs = 10, batch_size = 32, shuffle=True)